In [0]:
df = spark.read.options(header=True).csv('/Workspace/Users/anlrj@qq.com/creditcard.csv') 
df.show(5)

In [0]:
import pyspark.sql.functions as fn

bronze_df = (
    df.withColumn("_ingest_ts", fn.current_timestamp())
      .withColumn("_source_file", fn.col("_metadata.file_path") )
      .withColumn("_transaction_id", fn.monotonically_increasing_id()) 
)
bronze_df.show(5)


In [0]:
sql = "drop table if exists bronze_transactions"
spark.sql(sql)

In [0]:
bronze_df.write.format("delta").mode("overwrite").saveAsTable("bronze_transactions")

In [0]:
spark.sql("SELECT COUNT(*) FROM bronze_transactions").show()

In [0]:
sql = "select distinct amount from bronze_transactions where cast(amount as decimal(38, 10)) < 0 " 
df = spark.sql(sql) 
df.show()

In [0]:
sql = "desc bronze_transactions"
df = spark.sql(sql).filter("1=1") 
df.show(200)

In [0]:
sql = "desc bronze_transactions"
df = spark.sql(sql).filter("col_name like 'V%'")
df.select("col_name").show()

In [0]:

sql = """ 
select * from bronze_transactions 
where amount != '' 
and cast(amount as varchar(100)) != 'nan' 
and len(cast(amount as varchar(100))) > 0 
and cast(amount as decimal(38, 10)) >= 0

"""
df = spark.sql(sql)
df.write.format("delta").mode("overwrite").saveAsTable("silver_transactions")

In [0]:
sql = """
with cte as (
select _transaction_id, floor(_transaction_id / 2 ) customer_id  , Time, m, d , Amount , Class, _ingest_ts, _source_file from bronze_transactions 
unpivot 
 ( m for d in (V1
, V2 
, V3 
, V4 
, V5 
, V6 
, V7 
, V8 
, V9 
, V10 
, V11 
, V12 
, V13 
, V14 
, V15 
, V16 
, V17 
, V18 
, V19 
, V20))  upvt 
) 
select * from cte 
where amount != '' 
and cast(amount as varchar(100)) != 'nan' 
and len(cast(amount as varchar(100))) > 0 
and cast(amount as decimal(38, 10)) >= 0
""" 

df = spark.sql(sql) 
df.show(100)

